# ConversationEntityMemory

엔티티 메모리는 대화에서 특정 엔티티에 대한 주어진 사실을 기억합니다.

엔티티 메모리는 엔티티에 대한 정보를 추출하고(LLM 사용) 시간이 지남에 따라 해당 엔티티에 대한 지식을 축적합니다(역시 LLM 사용).


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


Entity 메모리를 효과적으로 사용하기 위하여, 제공되는 프롬프트를 사용합니다.


In [4]:
# Entity Memory를 사용하는 프롬프트 내용을 출력합니다.
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [5]:
# LLM 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# ConversationChain 을 생성합니다.
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
)

C:\Users\dukim\AppData\Local\Temp\ipykernel_3128\1337193518.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(


대화를 시작합니다.

입력한 대화를 바탕으로 `ConversationEntityMemory` 는 주요 Entity 정보를 별도로 저장합니다.


In [6]:
conversation.predict(
    input="테디와 셜리는 한 회사에서 일하는 동료입니다."
    "테디는 개발자이고 셜리는 디자이너입니다. "
    "그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
)

'테디와 셜리가 한 회사에서 일하는 동료이고, 테디는 개발자이고 셜리는 디자이너라는 정보를 알려주셨군요. 그들이 회사를 차릴 계획을 세우고 있다는 것은 미래에 두 사람이 협력하여 새로운 사업을 시작하려는 의지가 있는 것으로 보입니다. 이는 두 사람이 서로의 강점을 살려 협력하여 성공적인 사업을 이끌어 나갈 수 있는 좋은 기회가 될 것입니다. 새로운 도전에 대한 열정과 노력이 있으면 두 사람이 함께 성공을 거두리라고 믿습니다.'

Entity는 `memory.entity_store.store` 에서 확인할 수 있습니다.


In [7]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store

{'테디': '테디는 개발자이고, 셜리와 함께 자신들의 회사를 차릴 계획을 세우고 있습니다.',
 '셜리': '셜리는 한 회사에서 디자이너로 일하고 있으며, 테디와 함께 자신들의 회사를 차릴 계획을 세우고 있다.'}

In [9]:
conversation.predict(input="테디와 설리의 관계에 대해 요약해줘.")

'테디와 셜리는 한 회사에서 일하는 동료이고, 테디는 개발자이고 셜리는 디자이너입니다. 두 사람은 현재 자신들의 회사를 차릴 계획을 세우고 있습니다. 이는 두 사람이 서로의 강점을 살려 협력하여 성공적인 사업을 이끌어 나갈 수 있는 좋은 기회가 될 것으로 보입니다. 새로운 도전에 대한 열정과 노력이 있으면 두 사람이 함께 성공을 거두리라고 믿습니다.'